# First look at the data
* Text data are difficult to deal with because they are non-structured
* Reddit data is particularly dirty (lots of non-ascii characters, punctuations and incomplete sentences)

Here we have the raw data extracted by the scraper, along with the labels of the categories we assigned to them.

In [74]:
file = '../datasets/reddit_raw_with_labels.csv'

In [75]:
df = pd.read_csv(file)

In [76]:
df.head()

,title,score,num_comments,created_at,body,category_sl,majority_vote,label
0,Live conversation via phone,3,2,8/27/20 16:26,Hi guys I m quite confused. So i started this ...,live convo,live convo,live convo
1,What's a test with the most steps you've ever ...,6,9,8/27/20 11:19,"I got one with 57 steps a while back, and just...",bad test,bad test,bad test
2,Preparing for Live Conversation,1,4,8/27/20 10:44,Hi! Iâ€™ve been scheduled for a Live Test on A...,live convo,live convo,live convo
3,Mobile Test,0,8,8/27/20 6:50,Hey yâ€™all...Iâ€™m about to take a test on my...,mobile,mobile,mobile
4,Curious about message,1,7,8/27/20 6:49,&#x200B; [Anyone know what this is?](https://p...,other,other,other


Let's use the `label` column, which takes the top 100 labels of the `majority_vote`, and fill the rest with `category_sl`.

In [78]:
from sklearn.feature_extraction.text import CountVectorizer

In [79]:
text = df['title'][0]

In [81]:
vectorizer = CountVectorizer()

In [83]:
vectorizer.fit([text])

CountVectorizer()

In [84]:
vectorizer.vocabulary_

{'live': 1, 'conversation': 0, 'via': 3, 'phone': 2}

In [77]:
df_title_only = df[['title', 'label']]

0                            Live conversation via phone
1      What's a test with the most steps you've ever ...
2                        Preparing for Live Conversation
3                                            Mobile Test
4                                  Curious about message
                             ...                        
494    I now earn more with Usertesting than my full ...
495    iPhone Issue - Screener proceeds to next quest...
496       usertesting " Test in progress for over 2days"
497    Received UserTesting payment for first test, b...
498                              Tests for wrong country
Name: title, Length: 499, dtype: object

In [85]:
df['label'].value_counts()

other            144
bad test          55
screeners         52
ratings           43
recorder          42
live convo        41
mobile            37
no test           32
bug               29
payment           20
other company      1
Name: label, dtype: int64